In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [2]:
# ## 新的特征第二轮提取,尽量不和第一组重复,主要面向的最近一短时间的特诊,
# 1. 最近一天的[1,2,3,4,5,6,7,8,9]的次数.(时间基准,最后一次访问时间前一天[1h,3h,6h,12h,1,3,5,7,10,15]的访问次数,)

# 4. 统计所有的评价词语,将词分为正负情感两类,进行表示.


In [3]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [4]:
# 列名
column = ['userid']
for i in [0.042,0.125,0.25,0.5,1,3,5,7,15]:    
    # 计算[1-9]的各自的个数,
    for k in [1,2,3,4,5,6,7,8,9]:
        column.append('actionTimeLast'+str(i)+'Days'+str(k)+'Cnt')
    
    # [2-4]
    column.append('actionTimeLast'+str(i)+'Days24Cnt')
    
    # [5-9]
    column.append('actionTimeLast'+str(i)+'Days59Cnt')

    # [1-9]总数
    column.append('actionTimeLast'+str(i)+'Days19Cnt')


In [5]:
len(column)

109

In [6]:
userlist = list(set(df_train_action['userid']))

df_train_actionCnt = pd.DataFrame(columns=column)
cnt = 0
for user in userlist:
    cnt+=1
    if cnt % 3000 == 0:
        print(cnt)
    row = [user]
    t = df_train_action[df_train_action['userid'] == user]
    t = t.sort_values('actionTime')
    
    # 计算时间的前[1,3,5,7,15,* (60*60*24 = 86400 )]
    cur_time = t['actionTime'].values[-1]

    for i in [0.042,0.125,0.25,0.5,1,3,5,7,15]:    
        tmp = t[t['actionTime'] > cur_time - i * 86400]
        
        # 计算[1-9]的各自的个数,
        for k in [1,2,3,4,5,6,7,8,9]:
            row.append((tmp['actionType'] == k).sum())
        
        # [2-4]
        row.append(sum(row[2:5]))
        
        # [5-9]
        row.append(sum(row[5:10]))

        # [1-9]统计值
        row.append(int(len(tmp)))
            
    df_train_actionCnt = df_train_actionCnt.append(pd.DataFrame([row],columns=column),ignore_index=True)

df_train_data = pd.merge(df_train_data,df_train_actionCnt,how='left',on='userid')


3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000


In [7]:
userlist = list(set(df_test_action['userid']))

df_test_actionCnt = pd.DataFrame(columns=column)
cnt = 0
for user in userlist:
    cnt+=1
    if cnt % 3000 == 0:
        print(cnt)
    row = [user]
    t = df_test_action[df_test_action['userid'] == user]
    t = t.sort_values('actionTime')
    
    # 计算时间的前[1,3,5,7,15,* (60*60*24 = 86400 )]
    cur_time = t['actionTime'].values[-1]

    for i in [0.042,0.125,0.25,0.5,1,3,5,7,15]:    
        tmp = t[t['actionTime'] > cur_time - i * 86400]
        
        # 计算[1-9]的各自的个数,
        for k in [1,2,3,4,5,6,7,8,9]:
            row.append((tmp['actionType'] == k).sum())
        
        # [2-4]
        row.append(sum(row[2:5]))
        
        # [5-9]
        row.append(sum(row[5:10]))

        # [1-9]统计值
        row.append(int(len(tmp)))
            
    df_test_actionCnt = df_test_actionCnt.append(pd.DataFrame([row],columns=column),ignore_index=True)

df_test_data = pd.merge(df_test_data,df_test_actionCnt,how='left',on='userid')


3000
6000
9000


In [11]:
df_train_data.info()
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 55 columns):
actionTimeLast1Days6Cnt      40307 non-null object
actionTimeLast1Days7Cnt      40307 non-null object
actionTimeLast1Days8Cnt      40307 non-null object
actionTimeLast1Days9Cnt      40307 non-null object
actionTimeLast1Days24Cnt     40307 non-null object
actionTimeLast1Days59Cnt     40307 non-null object
actionTimeLast1Days19Cnt     40307 non-null object
actionTimeLast3Days1Cnt      40307 non-null object
actionTimeLast3Days2Cnt      40307 non-null object
actionTimeLast3Days3Cnt      40307 non-null object
actionTimeLast3Days4Cnt      40307 non-null object
actionTimeLast3Days5Cnt      40307 non-null object
actionTimeLast3Days6Cnt      40307 non-null object
actionTimeLast3Days7Cnt      40307 non-null object
actionTimeLast3Days8Cnt      40307 non-null object
actionTimeLast3Days9Cnt      40307 non-null object
actionTimeLast3Days24Cnt     40307 non-null object
actionTimeLast3Days59Cnt

In [12]:
df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_8.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_8.csv',index=False,index_label=False)
